# Final Project: Decide on global dual sourcing strategy

team: \
David Yang \
Jack Chen \
Joyce Wu

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('final project 2024.csv')

## Single Sourcing: Mexico

### Cumulative Average

In [17]:
def Mexico_cul_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance):
    cash_balance = initial_cash_balance
    inventory = 0

    for period in data['period']:

        # forecast # order 
        forecast = np.ceil(np.mean(data[data['period'] <= period]['demand']))  

        # revenue
        demand = data.loc[period-1, 'demand']
        rev = sales_price * np.minimum(inventory, demand)
        
        if inventory > demand:
            inventory -= demand
        else:
            inventory = 0

        cash_balance += rev
        
        # preparation of next period # in stock 
        if inventory >= forecast:
            pass
        else:
            order = (forecast - inventory)
            inventory += order  # purchase inventories
            cash_balance -= order * sourcing_cost  # cost of inventories

        # interest
        cash_balance *= (1 + interest_rate)
    return cash_balance

In [4]:
def Mexico_lagged_1(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance):
    cash_balance = initial_cash_balance
    inventory = 0

    for period in data['period']:

        # forecast # order 
        forecast = data.loc[period-1, 'demand']

        # revenue
        demand = data.loc[period-1, 'demand']
        rev = sales_price * np.minimum(inventory, demand)
        
        if inventory > demand:
            inventory -= demand
        else:
            inventory = 0

        cash_balance += rev
        
        # preparation of next period # in stock 
        if inventory >= forecast:
            pass
        else:
            inventory += forecast  # purchase inventories
            cash_balance -= forecast * sourcing_cost  # cost of inventories

        # interest
        cash_balance *= (1 + interest_rate)
    return cash_balance

In [5]:
def Mexico_rolling(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, window):
    cash_balance = initial_cash_balance
    inventory = 0
    window = window

    for period in data['period']:

        # forecast # order 
        if period >= 5:
            forecast = data.loc[period-window: period]['demand'].mean()
        else:
            forecast = data.loc[:period]['demand'].mean()

        # revenue
        demand = data.loc[period-1, 'demand']
        rev = sales_price * np.minimum(inventory, demand)
        
        if inventory > demand:
            inventory -= demand
        else:
            inventory = 0

        cash_balance += rev
        
        # preparation of next period # in stock 
        if inventory >= forecast:
            pass
        else:
            inventory += forecast  # purchase inventories
            cash_balance -= forecast * sourcing_cost  # cost of inventories

        # interest
        cash_balance *= (1 + interest_rate)
    return cash_balance

In [6]:
def Mexico_exponential(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha):
    cash_balance = initial_cash_balance
    inventory = 0
    last_forecast = 0

    for period in data['period']:

        # forecast # order 
        if period == 1:
            forecast = data.loc[data['period']==period, 'demand'].values[0]
        else:
            forecast = data.loc[data['period']==period, 'demand'].values[0] * alpha + (1 - alpha) * last_forecast
        
        last_forecast = forecast

        # revenue
        demand = data.loc[period-1, 'demand']
        rev = sales_price * np.minimum(inventory, demand)
        
        if inventory > demand:
            inventory -= demand
        else:
            inventory = 0

        cash_balance += rev
        
        # preparation of next period # in stock 
        if inventory >= forecast:
            pass
        else:
            inventory += forecast  # purchase inventories
            cash_balance -= forecast * sourcing_cost  # cost of inventories

        # interest
        cash_balance *= (1 + interest_rate)
    return cash_balance

In [18]:
Mexico_cul_avg(data, 10, 8, 0.01, 0)

1.2365738620601625e+47

In [8]:
Mexico_lagged_1(data, 10, 8, 0.01, 0)

1.2933300174659392e+47

In [9]:
Mexico_rolling(data, 10, 8, 0.01, 0, 1)

1.4340077521906562e+47

In [10]:
Mexico_exponential(data, 10, 8, 0.01, 0, 0.41)

1.3309104748619536e+47

In [11]:
alphas = np.arange(0.01, 1, 0.01)
max_balance = 0
max_alpha = 0.0

for alpha in alphas:
    if Mexico_exponential(data, 10, 8, 0.01, 0, alpha) > max_balance:
        max_balance = Mexico_exponential(data, 10, 8, 0.01, 0, alpha)
        max_alpha = alpha
print(f'Current {max_alpha}, Total Balanced: {max_balance}')

Current 0.41000000000000003, Total Balanced: 1.3309104748619536e+47


In [12]:
window_range = np.arange(1, 11, 1)
max_balance = 0
max_window = np.NaN

for window in window_range:
    if Mexico_rolling(data, 10, 8, 0.01, 0, window) > max_balance:
        max_balance = Mexico_rolling(data, 10, 8, 0.01, 0, window)
        max_window = window
print(f'Current {max_window}, Total Balanced: {max_balance}')

Current 1, Total Balanced: 1.4340077521906562e+47
